<a href="https://colab.research.google.com/github/SophieShin/MachineLearning/blob/main/Option_Pricing(%2Bprc_lmt).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 반드시 사본 저장 후 사본파일에서 실행해주세요.(파일/Drive에 사본저장)
#### 아래 '재생'버튼을 누르면서 진행하면 됩니다


In [1]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np

#### [CR20]화면에서 검증하고자 하는 상품의 이론가격 파라미터를 저장합니다
#### 결제월 : 전체
#### 선물옵션구분: 콜+풋
#### 이론가용도: 기준가용
#### 화면에서 프린트 버튼을 누른 뒤 엑셀로 저장하면 됩니다.
#### 엑셀로 저장할 때  'CSV' 형식으로 저장합니다.
#### 엑셀로 저장할 때 파라미터를 제외한 문구들(맨 위에 화면구성, 맨 아래에 주석 등)을 지우고 작업하는게 편합니다

In [4]:
# 파일 업로드
# 예제파일 2개(이론가파람_SK하이닉스, 상하한가) 파일을 https://github.com/SophieShin/MachineLearning/tree/main 에서 다운 받을 수 있습니다.

from google.colab import files

uploaded = files.upload()

Saving LG에너지솔루션.csv to LG에너지솔루션.csv


#### 아래 uploaded['이론가파람_SK하이닉스.csv'] 부분에서 한글 부분을 자신이 저장한 파일 이름으로 변경합니다

In [5]:
import io
df = pd.read_csv(io.BytesIO(uploaded['LG에너지솔루션.csv']), encoding='utf-8', thousands = ',')
df

,종목명,선물옵션구분,최종거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
0,"C 202308 350,000( 10)",콜,2023-08-10,350000.0,593000.0,16,3.75,0.0,0.921,243642.86
1,"C 202308 360,000( 10)",콜,2023-08-10,360000.0,593000.0,16,3.75,0.0,0.921,233714.78
2,"C 202308 370,000( 10)",콜,2023-08-10,370000.0,593000.0,16,3.75,0.0,0.921,223791.49
3,"C 202308 380,000( 10)",콜,2023-08-10,380000.0,593000.0,16,3.75,0.0,0.921,213932.21
4,"C 202308 390,000( 10)",콜,2023-08-10,390000.0,593000.0,16,3.75,0.0,0.921,204082.14
...,...,...,...,...,...,...,...,...,...,...
239,"P 202406 600,000( 10)",풋,2024-06-13,600000.0,593000.0,324,3.75,0.0,0.921,191499.08
240,"P 202406 640,000( 10)",풋,2024-06-13,640000.0,593000.0,324,3.75,0.0,0.921,216820.16
241,"P 202406 680,000( 10)",풋,2024-06-13,680000.0,593000.0,324,3.75,0.0,0.921,242970.85
242,"P 202406 720,000( 10)",풋,2024-06-13,720000.0,593000.0,324,3.75,0.0,0.921,272088.68


In [6]:
# null 값 제거
df= df.dropna(subset=['종목명'])

In [7]:
# 금리 단위 맞추기
df['금리(%)']= df['금리(%)']/100

In [8]:
# 풋옵션 이론가 산출 파라미터만 남기기
df2 = df[df['선물옵션구분'] == '풋']

In [9]:
# 콜옵션 이론가 산출 파라미터만 남기기
df = df[df['선물옵션구분'] == '콜']

#### 1. 콜옵션 이론가 구하기

In [10]:
# null값 제거
df = df.dropna(axis = 1)
df

,종목명,선물옵션구분,최종거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
0,"C 202308 350,000( 10)",콜,2023-08-10,350000.0,593000.0,16,0.0375,0.0,0.921,243642.86
1,"C 202308 360,000( 10)",콜,2023-08-10,360000.0,593000.0,16,0.0375,0.0,0.921,233714.78
2,"C 202308 370,000( 10)",콜,2023-08-10,370000.0,593000.0,16,0.0375,0.0,0.921,223791.49
3,"C 202308 380,000( 10)",콜,2023-08-10,380000.0,593000.0,16,0.0375,0.0,0.921,213932.21
4,"C 202308 390,000( 10)",콜,2023-08-10,390000.0,593000.0,16,0.0375,0.0,0.921,204082.14
...,...,...,...,...,...,...,...,...,...,...
117,"C 202406 600,000( 10)",콜,2024-06-13,600000.0,593000.0,324,0.0375,0.0,0.921,204142.92
118,"C 202406 640,000( 10)",콜,2024-06-13,640000.0,593000.0,324,0.0375,0.0,0.921,190773.59
119,"C 202406 680,000( 10)",콜,2024-06-13,680000.0,593000.0,324,0.0375,0.0,0.921,178233.87
120,"C 202406 720,000( 10)",콜,2024-06-13,720000.0,593000.0,324,0.0375,0.0,0.921,168661.29


In [12]:
# 이론가격을 구하기 위해 data 자료타입 변경

S = list(map(int,df['기초자산\n기준가격'].tolist()))
K = list(map(int,df['행사가격'].tolist()))
r = list(map (float,df['금리(%)'].tolist()))
div = list(map(float,df['배당  현재가치'].tolist()))
sigma = list(map(float,df['변동성'].tolist()))
t = list(map(int,df['잔존일수'].tolist()))
theo_prc_sys = list(map(float,df['이론가격'].tolist()))
name = df['종목명'].tolist()

In [14]:
# 상하한가 계산을 위해 기초자산 가격을 가격제한폭으로 설정하고 자료타입 변경
upper_lmt =  df['기초자산\n기준가격']*1.3
lower_lmt =  df['기초자산\n기준가격']*0.7
upper_lmt = list(map(int,upper_lmt.tolist()))
lower_lmt = list(map(int,lower_lmt.tolist()))

In [15]:
# call 옵션 이론가격 구하는 코드

def call(s,k,R,T,Div, Sigma):
  u = np.exp(Sigma*np.sqrt(T/365/49))
  d = np.exp(-Sigma*np.sqrt(T/365/49))
  p = (np.exp(R *T/365/49) - d)/(u-d)
  q = 1-p
  y = []

  for i in range(50):
    C_i = max((s-Div) * u**i * d**(49-i) -k , 0)
    x_i = np.math.factorial(49)/(np.math.factorial(49-i)*np.math.factorial(i))
    p_i = p**i
    q_i = q**(49-i)
    y.append(C_i*x_i* p_i*q_i)

  return sum(y)*np.exp(-R*T/365)

In [16]:
# 모든 행사가격, 모든 월물에 대해 이론가격 구하는 코드
call_price_temp = []

for i in range(len(name)):
  s = S[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  price =  call(s,k,R,T,Div, Sigma)
  call_price_temp.append(price)


In [17]:
# 상한가 구하기
upper_lmt_temp = []

for i in range(len(name)):
  s = upper_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]*2

  upper_price =  call(s,k,R,T,Div, Sigma)
  upper_lmt_temp.append(upper_price)


In [18]:
# 하한가 구하기
lower_lmt_temp =[]

for i in range(len(name)):
  s = lower_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = 0.03

  lower_price = call(s,k,R,T,Div, Sigma)
  lower_lmt_temp.append(lower_price)

In [19]:
# 구한 이론가와 시스템에서 산출한 이론가 비교, 상하한가 표출

call_price = pd.Series(call_price_temp)
upper_limit = pd.Series(upper_lmt_temp)
lower_limit = pd.Series(lower_lmt_temp)
df['theo_price_검증용'] = call_price
df['시스템 검증 차이'] = df['이론가격'] - df['theo_price_검증용']
df['상한가'] = upper_limit
df['하한가'] = lower_limit

In [20]:
df

,종목명,선물옵션구분,최종거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가
0,"C 202308 350,000( 10)",콜,2023-08-10,350000.0,593000.0,16,0.0375,0.0,0.921,243642.86,243642.861998,-0.001998,422905.540160,65674.869841
1,"C 202308 360,000( 10)",콜,2023-08-10,360000.0,593000.0,16,0.0375,0.0,0.921,233714.78,233714.779197,0.000803,413263.413319,55691.294693
2,"C 202308 370,000( 10)",콜,2023-08-10,370000.0,593000.0,16,0.0375,0.0,0.921,223791.49,223791.488119,0.001881,403621.286478,45707.719546
3,"C 202308 380,000( 10)",콜,2023-08-10,380000.0,593000.0,16,0.0375,0.0,0.921,213932.21,213932.214469,-0.004469,394074.669055,35724.144398
4,"C 202308 390,000( 10)",콜,2023-08-10,390000.0,593000.0,16,0.0375,0.0,0.921,204082.14,204082.144581,-0.004581,384718.252201,25740.569251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,"C 202406 600,000( 10)",콜,2024-06-13,600000.0,593000.0,324,0.0375,0.0,0.921,204142.92,204142.922584,-0.002584,512730.975628,0.000000
118,"C 202406 640,000( 10)",콜,2024-06-13,640000.0,593000.0,324,0.0375,0.0,0.921,190773.59,190773.589834,0.000166,504963.055751,0.000000
119,"C 202406 680,000( 10)",콜,2024-06-13,680000.0,593000.0,324,0.0375,0.0,0.921,178233.87,178233.872338,-0.002338,497334.297904,0.000000
120,"C 202406 720,000( 10)",콜,2024-06-13,720000.0,593000.0,324,0.0375,0.0,0.921,168661.29,168661.287805,0.002195,489705.540058,0.000000


In [21]:
print('시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목')
df[df['시스템 검증 차이'].abs() > 1]

## 아래에 칼럼명만 나오면 1원 이상 차이나는 종목이 없다는 뜻입니다.

시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목


,종목명,선물옵션구분,최종거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가


#### 2. 풋옵션 이론가 구하기

In [22]:
# null값 제거
df2 = df2.dropna(axis = 1)
df2

,종목명,선물옵션구분,최종거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
122,"P 202308 350,000( 10)",풋,2023-08-10,350000.0,593000.0,16,0.0375,0.0,0.921,67.99
123,"P 202308 360,000( 10)",풋,2023-08-10,360000.0,593000.0,16,0.0375,0.0,0.921,123.48
124,"P 202308 370,000( 10)",풋,2023-08-10,370000.0,593000.0,16,0.0375,0.0,0.921,183.77
125,"P 202308 380,000( 10)",풋,2023-08-10,380000.0,593000.0,16,0.0375,0.0,0.921,308.07
126,"P 202308 390,000( 10)",풋,2023-08-10,390000.0,593000.0,16,0.0375,0.0,0.921,441.58
...,...,...,...,...,...,...,...,...,...,...
239,"P 202406 600,000( 10)",풋,2024-06-13,600000.0,593000.0,324,0.0375,0.0,0.921,191499.08
240,"P 202406 640,000( 10)",풋,2024-06-13,640000.0,593000.0,324,0.0375,0.0,0.921,216820.16
241,"P 202406 680,000( 10)",풋,2024-06-13,680000.0,593000.0,324,0.0375,0.0,0.921,242970.85
242,"P 202406 720,000( 10)",풋,2024-06-13,720000.0,593000.0,324,0.0375,0.0,0.921,272088.68


In [23]:
#인덱스 초기화
df2.reset_index(inplace = True)

In [24]:
df2

,index,종목명,선물옵션구분,최종거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
0,122,"P 202308 350,000( 10)",풋,2023-08-10,350000.0,593000.0,16,0.0375,0.0,0.921,67.99
1,123,"P 202308 360,000( 10)",풋,2023-08-10,360000.0,593000.0,16,0.0375,0.0,0.921,123.48
2,124,"P 202308 370,000( 10)",풋,2023-08-10,370000.0,593000.0,16,0.0375,0.0,0.921,183.77
3,125,"P 202308 380,000( 10)",풋,2023-08-10,380000.0,593000.0,16,0.0375,0.0,0.921,308.07
4,126,"P 202308 390,000( 10)",풋,2023-08-10,390000.0,593000.0,16,0.0375,0.0,0.921,441.58
...,...,...,...,...,...,...,...,...,...,...,...
117,239,"P 202406 600,000( 10)",풋,2024-06-13,600000.0,593000.0,324,0.0375,0.0,0.921,191499.08
118,240,"P 202406 640,000( 10)",풋,2024-06-13,640000.0,593000.0,324,0.0375,0.0,0.921,216820.16
119,241,"P 202406 680,000( 10)",풋,2024-06-13,680000.0,593000.0,324,0.0375,0.0,0.921,242970.85
120,242,"P 202406 720,000( 10)",풋,2024-06-13,720000.0,593000.0,324,0.0375,0.0,0.921,272088.68


In [25]:
# 이론가격을 구하기 위해 data 자료구조 변경

S = list(map(int,df2['기초자산\n기준가격'].tolist()))
K = list(map(int,df2['행사가격'].tolist()))
r = list(map (float,df2['금리(%)'].tolist()))
div = list(map(float,df2['배당  현재가치'].tolist()))
sigma = list(map(float,df2['변동성'].tolist()))
t = list(map(int,df2['잔존일수'].tolist()))
theo_prc_sys = list(map(float,df2['이론가격'].tolist()))
name = df2['종목명'].tolist()

In [27]:
# 상하한가 계산을 위해 기초자산 가격을 가격제한폭으로 설정하고 자료타입 변경
upper_lmt =  df['기초자산\n기준가격']*0.7
lower_lmt =  df['기초자산\n기준가격']*1.3
upper_lmt = list(map(int,upper_lmt.tolist()))
lower_lmt = list(map(int,lower_lmt.tolist()))

In [28]:
# put 옵션 이론가격 구하는 코드

def put(s,k,R,T,Div, Sigma):
  u = np.exp(Sigma*np.sqrt(T/365/49))
  d = np.exp(-Sigma*np.sqrt(T/365/49))
  p = (np.exp(R *T/365/49) - d)/(u-d)
  q = 1-p
  y = []

  for i in range(50):
    P_i = max(k -(s-Div)* u**i * d**(49-i)  , 0)
    x_i = np.math.factorial(49)/(np.math.factorial(49-i)*np.math.factorial(i))
    p_i = p**i
    q_i = q**(49-i)
    y.append(P_i*x_i* p_i*q_i)

  return sum(y)*np.exp(-R*T/365)

In [29]:
# 모든 행사가격, 모든 월물에 대해 이론가격 구하는 코드

put_price_temp = []

for i in range(len(name)):
  s = S[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  price =  put(s,k,R,T,Div,Sigma)
  put_price_temp.append(price)


In [30]:
# 상한가 구하기
upper_lmt_temp = []

for i in range(len(name)):
  s = upper_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]*2

  upper_price =  put(s,k,R,T,Div, Sigma)
  upper_lmt_temp.append(upper_price)

In [31]:
# 하한가 구하기
lower_lmt_temp =[]

for i in range(len(name)):
  s = lower_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = 0.03

  lower_price = put(s,k,R,T,Div, Sigma)
  lower_lmt_temp.append(lower_price)

In [32]:
# 구한 이론가와 시스템에서 산출한 이론가 비교, 상하한가 표출

call_price = pd.Series(put_price_temp)
upper_limit = pd.Series(upper_lmt_temp)
lower_limit = pd.Series(lower_lmt_temp)
df2['theo_price_검증용'] = call_price
df2['시스템 검증 차이'] = df2['이론가격'] - df2['theo_price_검증용']
df2['상한가'] = upper_limit
df2['하한가'] = lower_limit

In [33]:
df2

,index,종목명,선물옵션구분,최종거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가
0,122,"P 202308 350,000( 10)",풋,2023-08-10,350000.0,593000.0,16,0.0375,0.0,0.921,67.99,67.992158,-0.002158,31066.867436,0.000000e+00
1,123,"P 202308 360,000( 10)",풋,2023-08-10,360000.0,593000.0,16,0.0375,0.0,0.921,123.48,123.484504,-0.004504,35472.646489,0.000000e+00
2,124,"P 202308 370,000( 10)",풋,2023-08-10,370000.0,593000.0,16,0.0375,0.0,0.921,183.77,183.768574,0.001426,40083.544980,0.000000e+00
3,125,"P 202308 380,000( 10)",풋,2023-08-10,380000.0,593000.0,16,0.0375,0.0,0.921,308.07,308.070071,-0.000071,44694.443471,0.000000e+00
4,126,"P 202308 390,000( 10)",풋,2023-08-10,390000.0,593000.0,16,0.0375,0.0,0.921,441.58,441.575330,0.004670,49305.341962,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,239,"P 202406 600,000( 10)",풋,2024-06-13,600000.0,593000.0,324,0.0375,0.0,0.921,191499.08,191499.082550,-0.002550,393028.173369,0.000000e+00
118,240,"P 202406 640,000( 10)",풋,2024-06-13,640000.0,593000.0,324,0.0375,0.0,0.921,216820.16,216820.160464,-0.000464,426777.089144,3.916228e-14
119,241,"P 202406 680,000( 10)",풋,2024-06-13,680000.0,593000.0,324,0.0375,0.0,0.921,242970.85,242970.853633,-0.003633,460526.004920,1.075969e-05
120,242,"P 202406 720,000( 10)",풋,2024-06-13,720000.0,593000.0,324,0.0375,0.0,0.921,272088.68,272088.679763,0.000237,494274.920695,7.027785e-01


In [34]:
print('시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목')
df2[df2['시스템 검증 차이'].abs() > 1]

## 아래에 칼럼명만 나오면 1원 이상 차이나는 종목이 없다는 뜻입니다.

시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목


,index,종목명,선물옵션구분,최종거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가
